### Libraries:

In [9]:
import pandas as pd

# path_to_src = Replace to your path
# import sys; sys.path += [path_to_src]

from data.acquisition import get_weather_data, get_bank_holiday_data
from features.preprocessing import past_events, prepare_delays, distances_between_hospitals, rolling_sums_and_means, enrich_data, situation_at_all_hospitals

### Reading in the fake data for Assignments, Incidents and Patient Transfers System (PTS)

In [10]:
df_arrived = pd.read_parquet('../outputs/arrived.parquet')
df_incidents = pd.read_parquet('../outputs/incidents.parquet')
df_pts = pd.read_parquet('../outputs/pts.parquet')

### Preparing a dataframe for past events

The past events function takes the Assignments, Incidents and PTS dataframes and merges them together to give an overall representation of the hospital at each timestamp. We output a dataframe that describes the following: 
- hospital
- time
- number of ambulances arriving
- number of specific skills missing from ambulance arrivals (e.g. missing_skill_A, missing_skill_B)
- idea of flow in and out of the hospital using admissions, discharges and transfers (PTS)
- the mean handover delays
- the number of each age band arriving in ambulances
- the number of patients arriving with each responding priority
- the number of patients being transported by the ambulances
- the number of ambulances leaving the queue

For example, at a specifc time there will be a 1 in missing_skill_A if the no crew members on that ambulance have that skill.

In [11]:
df_past_events = past_events(df_arrived, df_incidents, df_pts)
df_past_events.head()

,hospital,time,num_ambulance_arrivals,missing_skill_A,missing_skill_B,missing_skill_D,missing_skill_C,missing_skill_E,flow,past_delay_mins,...,priority_4,priority_5,priority_6,priority_7,priority_8,priority_9,1_patient_transported,2_patients_transported,3_patients_transported,num_ambulance_depatures
0,hospital_A,1953-01-12 17:39:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,hospital_A,1953-01-12 21:27:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,hospital_A,1953-01-12 22:56:38,1.0,1.0,0.0,1.0,0.0,1.0,NaN,160623.69,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
3,hospital_A,1953-01-13 00:18:17,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hospital_A,1953-01-13 02:02:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [12]:
# Preparing delays dataframe and outputting the first five rows
delays = prepare_delays(df_arrived)
delays.head()

,time_plus_3,hospital,handover_delay_mins,time_plus_10,time_plus_24
0,1953-02-18 10:23:56,hospital_A,63586.14,1953-02-18 10:23:56,1953-02-18 10:23:56
1,1953-07-10 19:59:38,hospital_E,0.00,1953-07-10 19:59:38,1953-07-10 19:59:38
2,1953-02-04 21:14:28,hospital_A,0.00,1953-02-04 21:14:28,1953-02-04 21:14:28
3,1953-09-18 02:43:55,hospital_A,166719.69,1953-09-18 02:43:55,1953-09-18 02:43:55
4,1953-08-16 20:08:26,hospital_D,0.00,1953-08-16 20:08:26,1953-08-16 20:08:26


### Preparing distances dataframe

Using a function from preprocessing.py which calculates the Euclidean distance between each of the 11 hospitals to be used to see whether the number of ambulances arriving at hospitals nearby has an impact on the situation at the current hospital.

In [13]:
distances = distances_between_hospitals(df_arrived)
distances.head()

,hospital,hospital_other,distance
0,hospital_A,hospital_A,0.000000
1,hospital_A,hospital_E,616679.353909
2,hospital_A,hospital_D,432728.963990
3,hospital_A,hospital_C,904816.466515
4,hospital_A,hospital_B,112358.527625


### Preparing three dataframes representing the situation at all hospitals. 

The three dataframes being constructed are to be used as training data; one for the prediction three hours ahead, one for the prediction ten hours ahead and one for the prediction twenty four hours ahead. These dataframes are constructed using the situation at all hospitals function from preprocessing.py. This function combines rolling aggregations of the past events dataframe aligned to the timestamps of the handover delay.

In [14]:
df_for_three_hour_pred = situation_at_all_hospitals(delays, distances, df_past_events, 3)
df_for_ten_hour_pred = situation_at_all_hospitals(delays, distances, df_past_events, 10)
df_for_twenty_four_hour_pred = situation_at_all_hospitals(delays, distances, df_past_events, 24)

### Enriching the situational data with weather data and bank holiday data

Adding on the following information:
- maximum temperature
- minimum temperature
- rainfall
- whether there is a bank holiday or not on that date (Yes/No)

In [15]:
df_for_three_hour_pred = enrich_data(df_for_three_hour_pred, 1950)
df_for_ten_hour_pred = enrich_data(df_for_ten_hour_pred, 1950)
df_for_twenty_four_hour_pred = enrich_data(df_for_twenty_four_hour_pred, 1950)

In [19]:
# Example output of the dataframe for the three hour prediction
df_for_three_hour_pred.head()

,delay_id,time_plus_3,hospital,handover_delay_mins,time_plus_10,time_plus_24,missing_skill_A,missing_skill_B,missing_skill_C,missing_skill_D,...,time,date,day_of_week,month,year,hour_of_day,bank_holiday,tmax,tmin,rain
0,6901.0,1953-01-12 22:56:38,hospital_A,160623.69,1953-01-12 22:56:38,1953-01-12 22:56:38,NaN,NaN,NaN,NaN,...,1953-01-12 22:56:38,1953-01-12,Monday,1,1953,22,No,5.4,0.8,21.8
1,8451.0,1953-01-13 10:11:13,hospital_A,42447.04,1953-01-13 10:11:13,1953-01-13 10:11:13,NaN,NaN,NaN,NaN,...,1953-01-13 10:11:13,1953-01-13,Tuesday,1,1953,10,No,5.4,0.8,21.8
2,4540.0,1953-01-13 11:29:56,hospital_A,0.00,1953-01-13 11:29:56,1953-01-13 11:29:56,NaN,NaN,NaN,NaN,...,1953-01-13 11:29:56,1953-01-13,Tuesday,1,1953,11,No,5.4,0.8,21.8
3,7717.0,1953-01-13 21:13:41,hospital_A,17360.31,1953-01-13 21:13:41,1953-01-13 21:13:41,NaN,NaN,NaN,NaN,...,1953-01-13 21:13:41,1953-01-13,Tuesday,1,1953,21,No,5.4,0.8,21.8
4,5157.0,1953-01-14 02:42:13,hospital_A,0.00,1953-01-14 02:42:13,1953-01-14 02:42:13,0.0,0.0,1.0,1.0,...,1953-01-14 02:42:13,1953-01-14,Wednesday,1,1953,2,No,5.4,0.8,21.8


### Saving the dataframes as `.parquet`:

In [11]:
# Saving the three dataframes as a parquet file to your directory
df_for_three_hour_pred.to_parquet('../outputs/data_for_three_hour_pred.parquet')
df_for_ten_hour_pred.to_parquet('../outputs/data_for_ten_hour_pred.parquet')
df_for_twenty_four_hour_pred.to_parquet('../outputs/data_for_twenty_four_hour_pred.parquet')

## END.

___
___